#### Exercise problem

$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} x_i + y && \\
\text{subject to  }
& \sum_{i \in I} x_i + y \le 20\\
& 15 \le x_i + y 
&& \qquad \forall i \in I \\
& \sum_{i \in I} i*x_i \ge 10\\
& 30 \le x_5 + 2y 
\end{alignat*}
$$

In [1]:
import pyomo.environ as pe
import pyomo.opt as po

In [2]:
m = pe.ConcreteModel()
solver = po.SolverFactory("glpk")

In [3]:
m.I = pe.RangeSet(1,5)

m.x = pe.Var(m.I, domain=pe.NonNegativeIntegers)
m.y = pe.Var(domain=pe.PositiveReals)


m.cons1 = pe.Constraint(
    expr=(sum([m.x[i] for i in m.I]) + m.y <= 20)
)
m.cons2 = pe.Constraint(
    expr=(sum([m.x[i]*i for i in m.I]) >= 10)
)

def _cons(m, i):
    return m.x[i] + m.y >= 15
m.cons = pe.Constraint(m.I, rule=_cons)

m.cons3 =  pe.Constraint(
    expr= (30 <= m.x[5] + 2*m.y)
)

expr = sum([m.x[i] for i in m.I]) + m.y
m.objective = pe.Objective(sense=pe.minimize, expr=expr)
m.pprint()

1 RangeSet Declarations
    I : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]

2 Var Declarations
    x : Size=5, Index=I
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :  None : False :  True : NonNegativeIntegers
          2 :     0 :  None :  None : False :  True : NonNegativeIntegers
          3 :     0 :  None :  None : False :  True : NonNegativeIntegers
          4 :     0 :  None :  None : False :  True : NonNegativeIntegers
          5 :     0 :  None :  None : False :  True : NonNegativeIntegers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : PositiveReals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3] + x[4] + x[5] + y

4 Constraint Declarations
    cons : Size=5, In

In [4]:
results = solver.solve(m)
for i in m.I:
    print(f"x{i}: {pe.value(m.x[i])}")
print(f"y: {pe.value(m.y)}")
print('Obj: ', pe.value(m.objective))

x1: 0.0
x2: 0.0
x3: 0.0
x4: 0.0
x5: 2.0
y: 15.0
Obj:  17.0
